# ALeRCE watchlists, list of oids

```Author: Alejandra Muñoz Arancibia```

ALeRCE notebook

Read a list of object identifiers, connect to the database and check which of the objects have recent detections (date criterion can be changed). Save the list of objects that meet the date criterion to a csv file, including their information from the ``objects`` table

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
from astropy.time import Time
import psycopg2
from datetime import date, datetime, timedelta
import json

Define directory and input file names

In [3]:
dir_watchlists = 'example_tables/' # can be changed to preferred directory
namefil = dir_watchlists+'sources_example.csv' # can be changed to preferred file

Input csv file structure: 1 line header, first column has oid, comments start with '#'

In [4]:
objs = pd.read_csv(namefil, sep=',', comment='#', skip_blank_lines=True)
objs['oid'] = objs['oid'].str.replace(' ', '') # remove any space in oid
#display(objs.head())
print('Number of oids:', len(objs))

objs_uniq = pd.unique(objs['oid'])
objs = pd.DataFrame(objs_uniq, columns=['oid'])
nobjs = len(objs)

display(objs)
#display(objs.head())
print('Number of unique oids:', nobjs)

Number of oids: 7


,oid
0,ZTF20abyptpc
1,ZTF20acgigfo
2,ZTF20abxgxjd
3,ZTF18acurdfi
4,ZTF18absggyi
5,ZTF18abaqycq
6,ZTF19aascajb


Number of unique oids: 7


Connect to DB

In [5]:
dir_dbfile = '../usecases/' # can be changed to preferred directory
credentials_file = dir_dbfile+'alercereaduser_v2.json' # online data
#credentials_file = dir_dbfile+'alercereaduser_v3.json' # new db

with open(credentials_file) as jsonfile:
    params = json.load(jsonfile)["params"]

conn = psycopg2.connect(dbname=params['dbname'], user=params['user'], host=params['host'],
                        password=params['password'])

Check that all oids are found in the DB

In [6]:
query = '''
SELECT
    *
FROM
    objects
WHERE
    oid in (%s)
''' % ",".join(["'%s'" % oid for oid in objs['oid']])
objs_indb = pd.read_sql_query(query, conn)
objs_indb.set_index('oid', inplace=True)
nobjs_indb = len(objs_indb)

display(objs_indb)
#display(objs_indb.head())
print('Number of oids found in db:', nobjs_indb)

if nobjs_indb!=nobjs:
    print('Number of unique oids:', nobjs, 'Problem!')

,nobs,mean_magap_g,mean_magap_r,median_magap_g,median_magap_r,max_magap_g,max_magap_r,min_magap_g,min_magap_r,sigma_magap_g,...,deltajd,lastmjd,firstmjd,period,catalogid,classxmatch,classrf,pclassrf,pclassearly,classearly
oid,,,,,,,,,,,,,,,,,,,,,
ZTF18abaqycq,214,19.261053,16.064374,19.562508,16.240995,22.017512,18.806114,16.840645,14.190115,1.139188,...,873.677072,59157.078148,58283.401076,None,None,None,4,0.988036,0.995911,20
ZTF18absggyi,166,15.693993,15.331590,15.862771,15.439172,16.128538,15.657760,14.761267,14.694319,0.346296,...,817.916319,59180.166528,58362.250208,None,None,None,5,0.956160,0.988680,20
ZTF18acurdfi,137,17.580142,16.957054,17.652945,17.070848,18.107182,17.857178,16.648235,16.076652,0.352519,...,718.064178,59179.525139,58461.460961,None,None,None,8,0.715444,0.821519,19
ZTF19aascajb,99,19.917325,19.805096,19.866660,19.811933,21.022226,20.458278,19.552661,19.522846,0.233495,...,192.629340,59039.177326,58846.547986,None,None,None,24,0.411384,0.541139,18
ZTF20abxgxjd,42,19.222247,18.822616,19.103800,18.769100,20.999901,20.016899,18.410400,18.175301,0.765393,...,46.927280,59137.112199,59090.184919,None,None,None,10,0.282000,0.633775,19
ZTF20abyptpc,73,15.991961,15.697200,15.604600,15.397000,17.747999,17.818800,14.616600,14.634500,1.211162,...,79.793981,59178.154525,59098.360544,None,None,None,12,0.256896,0.628658,19
ZTF20acgigfo,18,18.973100,18.481600,19.082701,18.345301,19.822500,20.207899,17.432501,16.840599,0.931969,...,45.852905,59170.141262,59124.288357,None,None,None,10,0.394992,0.408832,19


Number of oids found in db: 7


Find current datetime in local and UTC formats

In [7]:
date_now_local = datetime.now()
offset_utc = 3. # use 3 (4) for Chile Summer (Winter) time, change for your time zone
date_now_utc = date_now_local+timedelta(hours=offset_utc)

date_now_local = date_now_local.strftime('%Y-%m-%dT%H:%M:%S')
date_now_utc = date_now_utc.strftime('%Y-%m-%dT%H:%M:%S')

date_now_local, date_now_utc

('2020-11-27T16:32:31', '2020-11-27T19:32:31')

Define date criterion. Example: last 10 days (in MJD)

In [8]:
ndays_lim = 10 # can be changed to preferred date criterion

date_now_utc = Time(date_now_utc, format='isot', scale='utc')
date_lim = Time(date_now_utc, format="isot", scale="utc").mjd
date_lim = date_lim-ndays_lim

date_lim

59170.814247685186

Apply date criterion

In [9]:
query = '''
SELECT
    *
FROM
    objects
WHERE
    lastmjd >= %s
    AND oid in (%s)
''' % (date_lim, ','.join(["'%s'" % oid for oid in objs['oid']]))
objs_indb = pd.read_sql_query(query, conn)
objs_indb.set_index('oid', inplace=True)

display(objs_indb.head())
print('Number of oids found in db that match criterion:', len(objs_indb))

,nobs,mean_magap_g,mean_magap_r,median_magap_g,median_magap_r,max_magap_g,max_magap_r,min_magap_g,min_magap_r,sigma_magap_g,...,deltajd,lastmjd,firstmjd,period,catalogid,classxmatch,classrf,pclassrf,pclassearly,classearly
oid,,,,,,,,,,,,,,,,,,,,,
ZTF18absggyi,166,15.693993,15.331590,15.862771,15.439172,16.128538,15.657760,14.761267,14.694319,0.346296,...,817.916319,59180.166528,58362.250208,None,None,None,5,0.956160,0.988680,20
ZTF18acurdfi,137,17.580142,16.957054,17.652945,17.070848,18.107182,17.857178,16.648235,16.076652,0.352519,...,718.064178,59179.525139,58461.460961,None,None,None,8,0.715444,0.821519,19
ZTF20abyptpc,73,15.991961,15.697200,15.604600,15.397000,17.747999,17.818800,14.616600,14.634500,1.211162,...,79.793981,59178.154525,59098.360544,None,None,None,12,0.256896,0.628658,19


Number of oids found in db that match criterion: 3


Sort dataframe by last MJD where there are detections for each object, starting with most recent

In [10]:
col_to_sort = 'lastmjd' # can be changed to preferred column
ascending = False # if True, sorts from smallest to largest - can be changed to preferred sort

objs_indb_sort = objs_indb.sort_values(by=[col_to_sort], ascending=ascending)
objs_indb_sort.head(10)

,nobs,mean_magap_g,mean_magap_r,median_magap_g,median_magap_r,max_magap_g,max_magap_r,min_magap_g,min_magap_r,sigma_magap_g,...,deltajd,lastmjd,firstmjd,period,catalogid,classxmatch,classrf,pclassrf,pclassearly,classearly
oid,,,,,,,,,,,,,,,,,,,,,
ZTF18absggyi,166,15.693993,15.331590,15.862771,15.439172,16.128538,15.657760,14.761267,14.694319,0.346296,...,817.916319,59180.166528,58362.250208,None,None,None,5,0.956160,0.988680,20
ZTF18acurdfi,137,17.580142,16.957054,17.652945,17.070848,18.107182,17.857178,16.648235,16.076652,0.352519,...,718.064178,59179.525139,58461.460961,None,None,None,8,0.715444,0.821519,19
ZTF20abyptpc,73,15.991961,15.697200,15.604600,15.397000,17.747999,17.818800,14.616600,14.634500,1.211162,...,79.793981,59178.154525,59098.360544,None,None,None,12,0.256896,0.628658,19


Show Explorer link for each object. Allows to see the light curve, stamps, Aladin images, etc (useful for small lists)

In [11]:
for oid in objs_indb_sort.index:
    display(HTML("<a href='http://alerce.online/object/%s' target=\"_blank\"> %s <a>" % (oid, oid))) # online data
    #display(HTML("<a href='http://dev.alerce.online/object/%s' target=\"_blank\"> %s <a>" % (oid, oid))) # new Explorer

Save dataframe to csv file, file name will include the current datetime (defined above) in UTC format and date criterion

In [12]:
dir_out = 'out/' # can be changed to preferred directory

namein = namefil.replace(dir_watchlists, '')
namein = namein.replace('.csv', '_out_')

namefil2 = dir_out+namein+str(date_now_utc).replace(':', '_')+'_last'+str(int(ndays_lim))+'days.csv' # can be changed to preferred file name
objs_indb_sort.to_csv(namefil2)

print('Saved to file:', namefil2)

Saved to file: out/sources_example_out_2020-11-27T19_32_31.000_last10days.csv


Close connection to DB

In [13]:
conn.close()